In [1]:
from os import chdir
chdir('/home/shoaib/Thesis/dmdt_Analysis/')
from dmdt_functions import get_differenciation, get_2Dhistogram

In [2]:
from pandas import read_parquet, DataFrame, concat
from numpy import linspace

In [3]:
filtered_lightcurves_file = "/home/shoaib/PSChallenge/filtered_lightcurves.parquet"
original_features_by_oid = "/home/shoaib/PSChallenge/original_features_by_oid.csv"

xbins, ybins = 51, 51
# These files do'nt exist yet, they are save paths for later in the notebook
dmdt_by_name_band_50x50 = '/home/shoaib/Thesis/dmdt_Analysis/Histograms/dmdt_by_Name_Band_50x50.pkl'

In [4]:
### By experimentation we know that these are good bins
log_dt_bins = linspace(-4, 3.5, xbins)
dm_bins = linspace(-0.85, 0.8, ybins)

all_lightcurves = read_parquet(filtered_lightcurves)
all_lightcurves = all_lightcurves.sample(frac=0.1)

In [5]:
### Looping now
grouped = all_lightcurves.groupby(['name', 'band'])
histogram_dict_list = []
num_total_groups = len(grouped)
count = 0

for (name, band), df in grouped:
    count += 1
    print(f'{count}\t/ {num_total_groups} objects in their bands done.')
    
    df = df.sort_values(by='mjd')
    times, mags = df['mjd'].to_numpy(), df['mag'].to_numpy()
    dtimes, dmags = get_differenciation(times=times, magnitudes=mags)
    hist, _junk, _junk = get_2Dhistogram(dtimes=dtimes, dmagnitudes=dmags, dt_bins=log_dt_bins, dm_bins=dm_bins, normalise=False, scale_factor=False)
    
    if hist.sum() == 0:
        continue
    
    hist_normalised = hist / hist.sum()
    object_type = df['type'].iloc[0]
    n_good_det = len(df)
    
    histogram_dict = {'name': name, 'type': object_type, 'band': band, 'n_good_det': n_good_det,
                      'histogram': hist, 'histogram_normalised': hist_normalised}
    histogram_dict_list.append(histogram_dict)
    

1	/ 63602 objects in their bands done.
2	/ 63602 objects in their bands done.
3	/ 63602 objects in their bands done.
4	/ 63602 objects in their bands done.
5	/ 63602 objects in their bands done.
6	/ 63602 objects in their bands done.
7	/ 63602 objects in their bands done.
8	/ 63602 objects in their bands done.
9	/ 63602 objects in their bands done.
10	/ 63602 objects in their bands done.
11	/ 63602 objects in their bands done.
12	/ 63602 objects in their bands done.
13	/ 63602 objects in their bands done.
14	/ 63602 objects in their bands done.
15	/ 63602 objects in their bands done.
16	/ 63602 objects in their bands done.
17	/ 63602 objects in their bands done.
18	/ 63602 objects in their bands done.
19	/ 63602 objects in their bands done.
20	/ 63602 objects in their bands done.
21	/ 63602 objects in their bands done.
22	/ 63602 objects in their bands done.
23	/ 63602 objects in their bands done.
24	/ 63602 objects in their bands done.
25	/ 63602 objects in their bands done.
26	/ 6360

/home/shoaib/Thesis/dmdt_Analysis/dmdt_functions.py:33: RuntimeWarning: divide by zero encountered in log10
  dt = log10(dt)


145	/ 63602 objects in their bands done.
146	/ 63602 objects in their bands done.
147	/ 63602 objects in their bands done.
148	/ 63602 objects in their bands done.
149	/ 63602 objects in their bands done.
150	/ 63602 objects in their bands done.
151	/ 63602 objects in their bands done.
152	/ 63602 objects in their bands done.
153	/ 63602 objects in their bands done.
154	/ 63602 objects in their bands done.
155	/ 63602 objects in their bands done.
156	/ 63602 objects in their bands done.
157	/ 63602 objects in their bands done.
158	/ 63602 objects in their bands done.
159	/ 63602 objects in their bands done.
160	/ 63602 objects in their bands done.
161	/ 63602 objects in their bands done.
162	/ 63602 objects in their bands done.
163	/ 63602 objects in their bands done.
164	/ 63602 objects in their bands done.
165	/ 63602 objects in their bands done.
166	/ 63602 objects in their bands done.
167	/ 63602 objects in their bands done.
168	/ 63602 objects in their bands done.
169	/ 63602 obje

In [6]:
hist_df = DataFrame(histogram_dict_list)

In [7]:
grouped = hist_df.groupby('name')

for name, df in grouped:
    if len(df) < 2:
        continue
    
    hist = df['histogram'].sum()
    hist_normalised = hist / hist.sum()
    n_good_det = df['n_good_det'].sum()
    object_type = df['type'].iloc[0]

    histogram_dict = {'name': name, 'type': object_type, 'band': 'combined', 'n_good_det': n_good_det,
                      'histogram': hist, 'histogram_normalised': hist_normalised}
    histogram_dict_list.append(histogram_dict)

In [8]:
hist_df = DataFrame(histogram_dict_list)
hist_df = hist_df.sort_values(by=['type', 'name', 'band'], ignore_index=True)

In [9]:
hist_df.reset_index(drop=True).to_pickle(dmdt_by_name_band_50x50)

In [10]:
hist_df

,name,type,band,n_good_det,histogram,histogram_normalised
0,1E 22044+468,A,combined,129,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,1E 22044+468,A,g,58,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,1E 22044+468,A,r,71,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,2E 1853.7+4452,A,combined,66,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,2E 1853.7+4452,A,g,16,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...,...,...,...
94225,J230323.47-100235.4,Seyfert_1,g,10,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
94226,J230323.47-100235.4,Seyfert_1,r,11,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
94227,J230845.60-091123.9,Seyfert_1,combined,40,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
94228,J230845.60-091123.9,Seyfert_1,g,18,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
